# Upload data

In [1]:
!pip install pytest

In [26]:
import pandas as pd
import os #for viewing HTML in web browser
import matplotlib.pyplot as plt

In [28]:
#Open df in the browser - easiest way to view all the data in this large dataset
def to_html(df: pd.DataFrame) -> str:
    '''
    Render the df in HTML. Return the full HTML address. You can paste the HTML address in the 
    address bar to see the df at any time.
    '''
    df.to_html('df_view.html')
    full_path = os.path.abspath('df_view.html')
    print(full_path)

In [30]:
#Load file
# work_filepath = "C:\Users\kylimcqueen\Downloads\all_nts_animals_baseline_tall_00.csv"
#mac_filepath = '/Users/kyli/Desktop/Neurotrauma/all_nts_animals_baseline_tall_00.csv'
filepath=("\\Users\\kylimcqueen\\Downloads\\all_nts_animals_postinjury_tall_00.csv")

In [32]:
#Check that we grabbed the filepath
print(filepath)

\Users\kylimcqueen\Downloads\all_nts_animals_postinjury_tall_00.csv


In [34]:
#Create df
df_original = pd.read_csv(filepath)

In [36]:
#Look at the df
to_html(df_original)

C:\Git\Sleep-Analysis\df_view.html


# Data Cleaning

## Remove excluded animals

We are only using data from animals in the sham vehicle and injured vehicle groups.

In [44]:
#Check for excluded animals
def whats_in_the_col(df: pd.DataFrame, target_name: str, col_name: str) -> int, list:
    '''
    Counts the number of cells in the specified column of the dataframe that don't 
    exactly match the string "Included".
    '''
    # Get values that don't match the target_name
    non_matching_mask: list = df[col_name] != target_name
    
    # Count how many cells don't match
    count: int = non_matching_mask.sum()
    
    # Get the actual values that don't match
    non_matching_values: list = df.loc[non_matching_mask, col_name].unique().tolist()
    
    return count, non_matching_values

## Handle missing values

Data is in [TIDY](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html) format. This means each row is an observation and each column is a variable. For each different cohort, there is a certain number of NaNs at the beginning and end of sleep percent column. This is because of the way the data was originally formatted in the Excel. To address missing values, we need to get rid of all the leading and lagging NaNs and keep in mind that the number of leading and lagging Nans is the same within each cohort but different between each cohort.

In [21]:
def trim_nan_edges_nested(df, cohort_col, mouse_col, target_col):
    trimmed_groups = []

    # Loop over each cohort
    for _, cohort_group in df.groupby(cohort_col):
        # Loop over each UniqueMouse within the cohort
        for _, mouse_group in cohort_group.groupby(mouse_col):
            mouse_group = mouse_group.reset_index(drop=True)
            valid_mask = mouse_group[target_col].notna()

            if valid_mask.any():
                first_valid_idx = valid_mask.idxmax()
                last_valid_idx = len(valid_mask) - valid_mask[::-1].idxmax()
                trimmed = mouse_group.iloc[first_valid_idx:last_valid_idx].copy()
                trimmed_groups.append(trimmed)

    # Combine all cleaned groups
    trimmed_df = pd.concat(trimmed_groups, ignore_index=True)
    return trimmed_df


In [ ]:
# Remove duplicates

In [ ]:
# Check and convert datatypes

In [ ]:
# Clean text data

In [ ]:
# Filter and select data